<a href="https://colab.research.google.com/github/tomiaracodes/final_project_ImagineMe/blob/main/635_ImagineMe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffusers transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [4]:
import torch
from diffusers import StableDiffusionPipeline
import pandas as pd
import os

# Setup
csv_path = "/content/character_metadata.csv"
embeddings_dir = "/content/embeddings"
output_dir = "/content/generated_characters"
os.makedirs(output_dir, exist_ok=True)

# Load CSV
df = pd.read_csv(csv_path)
# Clean up column names: lowercase, strip spaces, replace spaces with underscores
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

print(df.columns.tolist())  # Check if 'character_name' now shows up correctly



['book_title', 'author', 'isbn', 'character_name', 'character_id', 'character_description', 'token_name']


In [17]:
# Load SD pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2",
    torch_dtype=torch.float16
).to("cuda")

# Default negative prompt
DEFAULT_NEGATIVE = (
    "blurry, low quality, unrealistic, white man, asian man, doll-like, painting, 3D, extra fingers, watermark, nudity"
)

# Directory with embeddings
embeddings_dir = "/content/embeddings"

# Load all embeddings
from glob import glob
for path in glob(f"{embeddings_dir}/*.bin"):
    token = "<" + os.path.splitext(os.path.basename(path))[0] + ">"
    pipe.load_textual_inversion(path, token=token)
    print(f"Loaded: {token}")

# Image generation
def generate_image(row):
    token = row["token_name"]
    prompt = f"A realistic portrait of {token}, {row['character_description']}"
    image = pipe(prompt=prompt, negative_prompt=DEFAULT_NEGATIVE, num_inference_steps=50).images[0]

    filename = f"{row['character_id']}_{row['character_name'].replace(' ', '_')}.png"
    image_path = os.path.join(output_dir, filename)
    image.save(image_path)
    return image_path

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loaded: <african_woman>
Loaded: <african_man>


In [18]:
# all characters
for _, row in df.iterrows():
    print(f"Generating for {row['character_name']} ({row['token_name']})...")
    try:
        path = generate_image(row)
        print(f"Saved: {path}")
    except Exception as e:
        print(f"Error: {e}")

Generating for Obinze (<african_man>)...


  0%|          | 0/50 [00:00<?, ?it/s]

Saved: /content/generated_characters/obinze_ameri_young_Obinze.png
Generating for Obinze (<african_man>)...


  0%|          | 0/50 [00:00<?, ?it/s]

Saved: /content/generated_characters/obinze_ameri_grown_Obinze.png
Generating for Ifemelu (<african_woman>)...


...

Saved: /content/generated_characters/sal_gh_Salim.png
Generating for Zelie (<african_woman>)...


  0%|          | 0/50 [00:00<?, ?it/s]

Saved: /content/generated_characters/zel_cobb_Zelie.png
Generating for Tzain (<african_man>)...


  0%|          | 0/50 [00:00<?, ?it/s]

Saved: /content/generated_characters/tzain_cobb_Tzain.png


In [12]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16
).to("cuda")
# Upload your .safetensors or .bin LoRA file
# from google.colab import files
# uploaded = files.upload()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Example prompt:

How you load your new embeddings to another model:

In [2]:
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt

In [3]:
!pip show torch

Name: torch
Version: 2.6.0+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision


In [ ]:
model_id1 =
model_id2 =